# Minimization of multiple conformers against single target

In [1]:
from openff.toolkit import Molecule, Topology, ForceField
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.interchange import Interchange
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import *
from openmm.app import PDBFile

Read in input files:
- conformers: SDF with multiple conformers of the same ligann
- ligand: reference pose which is used to relax the protein initially
- receptor: protein without bound ligand

In [2]:
conformers = Molecule.from_file("conformers.sdf")
ligand_ref = Molecule.from_file("ligand.sdf")
receptor_path = 'receptor.pdb'

Build protein topology

In [3]:
top = Topology.from_pdb(receptor_path)
sage_ff14sb = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")
protein_intrcg = Interchange.from_smirnoff(force_field=sage_ff14sb,topology=top)

Build ligand topology using NAGL charges

In [4]:
ligand_topology = ligand_ref.to_topology()
NAGLToolkitWrapper().assign_partial_charges(ligand_ref, 'openff-gnn-am1bcc-0.1.0-rc.3.pt')
ligand_intrcg = sage_ff14sb.create_interchange(ligand_topology, charge_from_molecules=[ligand_ref])

Combine protein+ligand topologies

In [5]:
%env INTERCHANGE_EXPERIMENTAL=1
docked_intrcg = protein_intrcg.combine(ligand_intrcg)

env: INTERCHANGE_EXPERIMENTAL=1


/data/homezvol3/ekubinco/miniforge3/envs/ssbind/lib/python3.11/site-packages/openff/interchange/components/interchange.py:950: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return _combine(self, other)
/data/homezvol3/ekubinco/miniforge3/envs/ssbind/lib/python3.11/site-packages/openff/interchange/operations/_combine.py:25: InterchangeDeprecationWarning: The `handlers` attribute is deprecated. Use `collections` instead.
  "vdW" in interchange1.handlers
/data/homezvol3/ekubinco/miniforge3/envs/ssbind/lib/python3.11/site-packages/openff/interchange/operations/_combine.py:26: InterchangeDeprecationWarning: The `handlers` attribute is deprecated. Use `collections` instead.
  and "vdW" in interchange2.handlers
/data/homezvol3/ekubinco/miniforge3/envs/ssbind/lib/python3.11/site-packages/openff/interchan

Visualize the complex

In [6]:
w = docked_intrcg.visualize()
w.clear_representations()
w.add_representation(
    "licorice",
    radius=0.1,
    selection=[*range(protein_intrcg.topology.n_atoms)],
)
w.add_representation(
    "spacefill",
    selection=[*range(protein_intrcg.topology.n_atoms, docked_intrcg.topology.n_atoms)],
)
w

NGLWidget()

## Set up OpenMM simulation

In [7]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
platform = Platform.getPlatformByName("CPU")
openmm_simulation = docked_intrcg.to_openmm_simulation(integrator, platform=platform)

Here the PDB does not have a CRYST1 entry so default periodic box vectors are used

In [8]:
openmm_simulation.context.getState().getPeriodicBoxVectors()

Quantity(value=[Vec3(x=2.0, y=0.0, z=0.0), Vec3(x=0.0, y=2.0, z=0.0), Vec3(x=0.0, y=0.0, z=2.0)], unit=nanometer)

Check platform

In [9]:
openmm_simulation.context.getPlatform().getName()

'CPU'

## Relax protein against reference ligand
This speeds up subsequent minimization of conformers

In [11]:
openmm_simulation.minimizeEnergy()

# save protein positions for reference
new_positions = openmm_simulation.context.getState(getPositions=True).getPositions()
protein_pos = new_positions[0:protein_intrcg.topology.n_atoms]

## Minimize conformers and structure

In [12]:
for i,conformer in enumerate(conformers):
    new_positions = openmm_simulation.context.getState(getPositions=True).getPositions()
    new_positions[0:protein_intrcg.topology.n_atoms] = protein_pos
    new_positions[protein_intrcg.topology.n_atoms:docked_intrcg.topology.n_atoms] = conformer.to_topology().get_positions().to_openmm()
    openmm_simulation.context.setPositions(new_positions)
    openmm_simulation.minimizeEnergy()
    state = openmm_simulation.context.getState(getPositions=True, getEnergy=True)
    with open('result.pdb', 'a+') as output:
        PDBFile.writeModel(openmm_simulation.topology, state.getPositions(), output, modelIndex=i)

## Extract minimized conformers to SDF

In [13]:
import MDAnalysis as mda
from rdkit import Chem

u = mda.Universe("result.pdb")
elements = mda.topology.guessers.guess_types(u.atoms.names)
u.add_TopologyAttr("elements", elements)
atoms = u.select_atoms("resname UNK")

sdwriter = Chem.SDWriter("minimized_conformers.sdf")
for _ in u.trajectory:
    sdwriter.write(atoms.convert_to("RDKIT"))

/data/homezvol3/ekubinco/miniforge3/envs/ssbind/lib/python3.11/site-packages/MDAnalysis/converters/RDKit.py:473: UserWarning: No `bonds` attribute in this AtomGroup. Guessing bonds based on atoms coordinates
  warnings.warn(
